In [ ]:
# For the creation of the ImageJ gateway
import imagej
from scyjava import jimport # To import java classes

# For the ScijavaCommand decorator (TODO)
import functools

# jpype documentation: https://jpype.readthedocs.io/en/latest/
#jpype imports
import jpype
from jpype import JImplements, JOverride
# Pull in types
from jpype.types import *

# For example command
import time
from random import random

# PyImageJ initialisation

See https://github.com/imagej/pyimagej for more details about ways to initialise pyimagej. Some demo notebooks are available in https://github.com/imagej/i2k-2020-pyimagej.
* Here the most specific package is [`ch.epfl.biop:pyimagej-scijava-command`](https://github.com/NicoKiaru/PyImageJ-Notebooks) which contains [`PyCommandBuilder`](https://github.com/NicoKiaru/PyImageJ-Notebooks/blob/master/src/main/java/org/scijava/command/PyCommandBuilder.java), which is the class necessary to register a command from Python. It can also be used on the java side, but that's much less interesting.
* `net.imagej:imagej-legacy` is also included. I don't know why it's necessary.


In [ ]:
imagej_core_dep = 'net.imagej:imagej:2.1.0' 
scijava_command_dep = 'ch.epfl.biop:pyimagej-scijava-command:0.1.2'
legacy_dep = 'net.imagej:imagej-legacy:0.37.4'

ij = imagej.init([imagej_core_dep, legacy_dep, scijava_command_dep], headless=False)

In [ ]:
# Import scijava classes using jimport
Command = jimport('org.scijava.command.Command') # Just for the example, not necessary in the notebook
JPlugin = jimport('org.scijava.plugin.Plugin')
Parameter = jimport('org.scijava.plugin.Parameter')
ImagePlus = jimport('ij.ImagePlus')

In [ ]:
# Different class which can be used as input or output parameter
# points to java object
print(JInt)
print(JInt.class_)
print(JBoolean)
print(JBoolean.class_)
print(ImagePlus)
print(ImagePlus.class_)
print(JString)
print(JString.class_)

# Not sure I understand
print(type(JInt))

# The class of a class is Class, ok
print(type(JInt.class_))

# **ScijavaCommand** decorator
Decorator that registers a python CLASS containing a method named "run" as a scijava Command. This uses `PyCommandBuilder` which is in the java repo `ch.epfl.biop:pyimagej-scijava-command`. `PyCommandBuilder` allows to build and register a command fully programmatically without using any java annotation as java annotations are needed for 'easy' Scijavy Commands definition but these are not completely supported in JPype (https://github.com/jpype-project/jpype/issues/940)

See cells below for example usage

Notes: 
* this way of defining a command is probably not ideal if this has to be used from the python side also
* WIP: this decorator prints a lot of stuff in the process
* TODO : use functools to avoid hiding the function

In [ ]:
#org.scijava.command.PyCommandBuilder
PyCommandBuilder = jimport('org.scijava.command.PyCommandBuilder')

def ScijavaCommand(**kwargs):    
    
    print("- Registering scijava command "+kwargs['name'])
    
    def registerCommand(func): 
        # This class will be registered as a SciJava Command
        builder = PyCommandBuilder() # Java PyCommandBuilder
        
        # The name of the command - to avoid name conflicts, consider a 'virtual' class name with its package
        builder = builder.name(kwargs['name'])
        
        # Register all inputs
        print('- Inputs')
        for name, javaClass in kwargs['inputs'].items():
            print('\t', name,' : ', javaClass)
            builder = builder.input(name, javaClass)
            setattr(func, name, None) # declares empty input field 
        print('Inputs registered')
        
        # Register all outputs
        print('- Outputs')
        for name, javaClass in kwargs['outputs'].items():
            print('\t', name,' : ', javaClass)
            builder = builder.output(name, javaClass)
            setattr(func, name, None) # declares empty output field
        print('Outputs registered')
                
        # Wraps the run function - takes kwargs as input, returns outputs
        def wrapped_run(inner_kwargs):
            inner_object = func()
            print('Settings inputs...')
            print(inner_kwargs)
            for name, javaClass in kwargs['inputs'].items():
                #print(name)
                #print(str(inner_kwargs[name]))
                setattr(inner_object, name, inner_kwargs[name]) # sets inputs 
            print('Inputs set.')
            print('Running scijava command: '+kwargs['name'])
            inner_object.run() #I'm not sure this actually works
            print(kwargs['name']+' command execution done.')
            print('Fetching outputs...')
            outputs = {}
            for name, javaClass in kwargs['outputs'].items():
                outputs[name] = getattr(inner_object, name) # gets outputs
            print('Outputs set.')
            return JObject(outputs, JClass('java.util.Map')) # Returns output as a java HashMap
        
        # Sets the function in PyCommandBuilder:
        # Function<Map<String, Object>, Map<String, Object>> command
        builder = builder.function(wrapped_run)
        
        # Effectively registers this command to the ij context
        builder.create(kwargs['context']) 
        return func
    
    return registerCommand


In [ ]:
# Now we can start the ui of ImageJ
# look for and execute the command 'pyCommand.HelloCommand' in the search bar
ij.ui().showUI()

# pyCommand.HelloCommand can be recorded in ImageJ and executed using a command service, 
# either in groovy (or jython) scripting in ImageJ, or in this notebook as shown below

In [ ]:
# Example of registering a Scijava Command via the @ScijavaCommand decorator

@ScijavaCommand(context = ij.context(), # ij context needed
                name = 'pyCommand.HelloCommand', # name of this command, mind potential naming conflicts!
                inputs = {'name': JString, 'familiar': JBoolean}, # input name, input Java class, as dictionary
                outputs = {'greetings': JString}) # output name, output Java class, as dictionary
class MyPyCommand:        
    def run(self):
        if (self.familiar):
            self.greetings = 'Hi ' + str(self.name) + '!'
        else:
            self.greetings = 'Hello my dear ' + str(self.name) + '.'


In [ ]:
# Example of registering a Scijava Command via the @ScijavaCommand decorator

@ScijavaCommand(context = ij.context(), # ij context needed
                name = 'pyCommand.RandomWaitReturnTime', # name of this command, mind potential naming conflicts!
                inputs = {'commandTag': JString}, # input name, input Java class, as dictionary
                outputs = {'waitedTime': JDouble}) # output name, output Java class, as dictionary
class RandomWaitCommand:        
    def run(self):
        waitInS = random()*5
        print("Starting " + str(self.commandTag) + " T = "+str(waitInS))
        time.sleep(waitInS)
        print("Ending " + str(self.commandTag))
        self.waitedTime = JDouble(waitInS)

In [ ]:
# Example of registering a Scijava Command via the @ScijavaCommand decorator

@ScijavaCommand(context = ij.context(), # ij context needed
                name = 'pyCommand.SliceImage2D', # name of this command, mind potential naming conflicts!
                inputs = {'image_in': ImagePlus}, # input name, input Java class, as dictionary
                outputs = {'image_out': ImagePlus}) # output name, output Java class, as dictionary
class SliceCommand:        
    def run(self):
        img_python = ij.py.from_java(self.image_in)
        print(img_python.shape)
        print("TODO : slicing the image!")
        ij.py.show(img_python, cmap = 'gray')
        self.image_out = ij.py.to_java(img_python)


In [ ]:
ij.command().run('pyCommand.HelloCommand', True) # no args -> all inputs required, check the UI!

In [ ]:
ij.command().run('pyCommand.HelloCommand', True, 'name', 'Bob') # familiar still required

In [ ]:
module = ij.command().run('pyCommand.HelloCommand', True, 'name', 'Bob', 'familiar', True) # all args given


# nodule returns None: Weird : how can I retrieve the output ? Or maybe it's eaten by the GUI ?

In [ ]:
# THIS CELL WON'T RUN
# Idea, another way of registering of Command could be like this:

@ScijavaFunctionDecorator
def myCommand(name: JString, familiar:JBoolean)
    greetings = "Hello" + name 
    return {'greetings':greetings, another param as a dictionary}

# But I'm not sure the decorator can be aware of the 'output' signature, thus the decorator
# will have a hard time, (maybe impossible?) to figure out the outputs and their class

In [ ]:

waitInS = random()*5
print("Now Waiting "+str(waitInS))
time.sleep(waitInS)
print("Ze end!")